In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
dff=pd.read_csv("Bank_Churn.csv")

In [4]:
df.columns

Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
df.isna().sum()

CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
df.dtypes

CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [8]:
df=pd.read_csv("Bank_Churn.csv")
db_path =r"C:\Users\USER\Desktop\My Work\Database-SQLite\Bank.db"
conn =sqlite3.connect(db_path)
cursor =conn.cursor()
table = """
CREATE TABLE IF NOT EXISTS Clients_Profile (
    CustomerId INTEGER PRIMARY KEY,
    Surname TEXT,
    CreditScore INTEGER,
    Geography TEXT,
    Gender TEXT,
    Age INTEGER,
    Tenure INTEGER,
    Balance INTEGER,
    NumOfProducts INTEGER,
    HasCrCard INTEGER,
    IsActiveMember INTEGER,
    EstimatedSalary INTEGER,
    Exited INTEGER
);
"""
cursor.execute(table)
conn.commit()
print("Table 'Clients_Profile' created successfully")
print("===" * 20)
df.to_sql("Clients_Profile",conn,if_exists="replace",index=False)
print("Data uploaded to 'Clients_Profile' table successfully!")

Table 'Clients_Profile' created successfully
Data uploaded to 'Clients_Profile' table successfully!


#### a) Lets view the first 5 rows of the table

In [11]:
query ="SELECT* FROM Clients_Profile LIMIT 5"
df =pd.read_sql(query,conn)
display(df.head(5))

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### b) How many clients does the Bank serve?

In [25]:
cursor.execute("SELECT COUNT(DISTINCT CustomerId) FROM Clients_Profile")
distinct_count =cursor.fetchone()[0]
print(f"The bank serves {distinct_count} customers")

The bank serves 10000 customers


#### c) How many customers have Exited/Churned and whats the Churn Rate?

In [36]:
cursor.execute("SELECT * FROM Clients_Profile WHERE Exited = 1")
exited =len(cursor.fetchall())
print(f"Out of {distinct_count} customers, {exited} have exited the bank")
print()
print(f"Churn Rate: {((exited/10000) * 100):.2f}%")

Out of 10000 customers, 2037 have exited the bank

Churn Rate: 20.37%


#### d) Count of Customers by Geography?

In [29]:
query ="""
SELECT Geography, COUNT(*) as Customer_Count
FROM Clients_Profile
GROUP BY Geography
ORDER BY Customer_Count DESC
"""
df =pd.read_sql(query,conn)
display(df)

,Geography,Customer_Count
0,France,5014
1,Germany,2509
2,Spain,2477


In [31]:
dff.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
